In [ ]:
%pip install -Uq "unstructured[all-docs]" pillow lxml
%pip install -Uq chromadb tiktoken
%pip install -Uq langchain langchain-community langchain-openai langchain-groq
%pip install -Uq python_dotenv

Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
from dotenv import load_dotenv

load_dotenv()

False

In [ ]:
from unstructured.partition.pdf import partition_pdf

output_path = "C:/Users/al4k9/OneDrive/Desktop/RAG/RRMenu.pdf"

chunks = partition_pdf(
    filename=output_path,                       # only required parameter
    infer_table_structure=False,                # extract tables
    strategy="auto",                            # mandatory to infer tables, "high-res" if you extract tables
    languages=["eng"],
    # extract_image_block_types=["Image"],      # add 'Table' to list to extrac image of table
    # extract_image_block_output_dir= "path",   # if none, images and tables will be saved in base64


    # chunking groups related information | Useful for RAG
    chunking_strategy="by_title",               # or "basic" | Menu has clear titles
    max_characters=10000,                       # max size of chunk
    combine_text_under_n_chars=2000,            # comebine different elements when they are under 2000 characters
    new_after_n_chars=6000                      # start new part after 6000 characters
    )

In [6]:
print(len(chunks))

9


In [8]:
chunks[0].to_dict()

{'type': 'CompositeElement',
 'element_id': '43c2099fd4e91cc48e55834faccbbcbd',
 'text': 'GOURMET FIRE-GRILLED BURGERS\n\nALL BURGERS INCLUDE A BOTTOMLESS SIDE\n\nSteak Fries • Yukon Chips • Steamed Broccoli • Side Salad Sweet Potato Fries (1.79) • Garlic Fries (1.79)\n\nOUR SIGNATURES Burgers you won’t find anywhere else. 100% Fresh, Never Frozen Beef.\n\nTHE SOUTHERN CHARM BURGER® Brown sugar glaze, candied bacon, Whiskey River® BBQ Sauce, Cheddar, caramelized onions, lettuce and mayo on a toasted brioche bun. 14.99 cal 1190\n\nBLACK & BLEU\n\nSautéed and blackened portobello mushrooms, caramelized onions, creamy cheese sauce, Bleu cheese crumbles, lettuce and roasted garlic aioli on a toasted brioche bun. 15.39 cal 980\n\nWHISKEY RIVER® BBQ\n\nDIP IT. DUNK IT. DOUSE YOUR BURGER.\n\nWhiskey River® BBQ Sauce, crispy onion straws, Cheddar, lettuce, tomatoes and mayo on a toasted sesame bun. 13.79 cal 1140 THE MADLOVE BURGER A Cheddar-and-Parmesan crisp, Provolone, Swiss, jalapeño relis

In [ ]:
texts = []

for chunk in chunks:
    if "CompositeElement" in str(type((chunk))):
        texts.append(chunk)

[<unstructured.documents.elements.CompositeElement object at 0x000001709538E450>, <unstructured.documents.elements.CompositeElement object at 0x00000170953C0B90>, <unstructured.documents.elements.CompositeElement object at 0x00000170953C0050>, <unstructured.documents.elements.CompositeElement object at 0x000001709C003BD0>, <unstructured.documents.elements.CompositeElement object at 0x000001709C0003D0>, <unstructured.documents.elements.CompositeElement object at 0x000001709C6564E0>, <unstructured.documents.elements.CompositeElement object at 0x000001708D3AE990>, <unstructured.documents.elements.CompositeElement object at 0x000001709527B4D0>, <unstructured.documents.elements.CompositeElement object at 0x0000017095358F70>]


In [9]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [18]:
prompt_text = """
You are an assistant tasked with summarizing text.
Give a concise summary of the text.

Respond only with the summary, no additional comments.
Do not start your message by saying "Here is a summary" or anything like that.
Just give me the summary as it is.

Text chunk: {element}

"""

prompt = ChatPromptTemplate.from_template(prompt_text)
model = ChatGroq(temperature=0.5, model="llama-3.1-8b-instant")
summarize_chain = prompt | model | StrOutputParser()

GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch

# Load the model
model_name = "mosaicml/mpt-7b-instruct"  # free instruct model
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Create an offload folder
offload_folder = os.path.join(os.getcwd(), "offload")
os.makedirs(offload_folder, exist_ok=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    offload_folder=offload_folder
)

# Your custom prompt template
prompt_template = """
You are an assistant tasked with summarizing text.
Give a concise summary of the text.

Respond only with the summary, no additional comments.
Do not start your message by saying "Here is a summary" or anything like that.
Just give me the summary as it is.

Text chunk: {element}
"""

def summarize_text(element, max_new_tokens=200):
    """
    Summarize a single text chunk using a local LLM with the custom prompt.
    
    Args:
        element (str): Text chunk to summarize.
        max_new_tokens (int): Max tokens for the summary.
    
    Returns:
        str: The concise summary.
    """
    prompt = prompt_template.format(element=element)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary.strip()

summary = []
for chunk in texts:
    summary.append(summarize_text(chunk))
print(summary)


Loading checkpoint shards:   0%|          | 0/2 [00:10<?, ?it/s]


OSError: [Errno 28] No space left on device